In [1]:
import requests
import traceback
import base64
import json
import os
import time
import mysql.connector
from tqdm.notebook import tqdm
# from tqdm import tqdm
from requests.auth import HTTPBasicAuth
import argparse
import pandas as pd
import getpass

In [2]:
def checkUser(username):
    r = requests.get(apiaddress+'/rest/api/2/user/?username='+username+'&expand=groups,applicationRoles',auth=HTTPBasicAuth(apiuser, apipass))
    if r.status_code !=200:
        return None
    else:
        with open(os.path.join('collecteduser',username+'_user.json'), 'w') as json_file:  
            json.dump(r.json(), json_file)
        return r.json()

In [3]:
apiuser='lamogibeb'
apipass='1234567890'
apiaddress='http://issues.apache.org/jira'

In [4]:
df = pd.read_csv('invalid_username.csv')
len(df['username'])

2430

In [22]:
for index, row in tqdm(df.iterrows(),total = df.shape[0]):
    success = False
    if str(df.loc[index,'check']) != 'nan':
        continue
    while not success:
        try:
            df.loc[index,'check']= True if checkUser(row['username']) is not None else False
            time.sleep(.065)
            success = True
        except Exception:
            print(traceback.format_exc())
            time.sleep(10)
            success = False

In [27]:
df[(df['check']==False)].shape[0]

1177

In [28]:
df[(df['check']==True)].shape[0]

1253

In [30]:
df.rename(columns={'check':'isApacheID'},inplace=True)
df.to_csv('checkedusername.csv')